# Bootlets
Bootlets is a Python library that provides a set of template objects which render HTML code, so you never have to write any HTML again!  

The `bootlets.html` module contains generic classes for most HTML tags, and `bootlets.boots` contains several classes to aid creating [Bootstrap](https://getbootstrap.com/) components.

The templates can be subclassed and customised, or changed on the fly, to modify its' content or keyword arguments.

In [1]:
# Import some Notebook tools
from IPython.display import display, HTML, IFrame

# Define a function to display the HTML strings as HTML
def print_html(html):
    display(HTML(html))

## HTML

In [2]:
# Import the base template classes
from bootlets.html import *
from bootlets.boots import Container

`bootlets.html` contains classes for most (one day all) HTML tags (e.g. `Div()` for `<div>`)  
`bootlets.boots` contains more complex classes, including a `Container` to wrap several html, or other boots

By default when the template class is instanciated all `*args` are joined by a newline, and `**kwargs` are used as keywords for the HTML tag. The `draw()` method is used to return the generated HTML code. For example:

In [3]:
P('This is come content', 'some more content', class_='bold').draw()

'<p class="bold">This is come content\nsome more content</p>'

The HTML objects can be nested within one another, like you would in the HTML code:

In [4]:
class Container_(Container):
    def __repr__(self) -> str:
        s = self.__class__.__qualname__ + "("
        if self.args:
            s += ", ".join([arg.__repr__() for arg in self.args])
        if self.kwargs:
            if self.args:
                s += ", "
            s += ", ".join([f"{k}={v.__repr__()}" for k, v in self._kwargs.items()])
        s += ")"
        return s

In [5]:
html = Container_(
            H('Header 1', size=1),
            P('paragraph'),
            Div(P('seperate set of'), P('paragraphs', class_='bold')),
            UlList(*['List', 'of', 'things']),
            DlDict({'title': 'description', 
                             'another title': 'another description',
                             'nested list': UlList(*['one', 'two'])
                            })
        )

In [6]:
# Print the HTML generated
print(html.draw())

<h1 size="1">Header 1</h1>
<p>paragraph</p>
<div><p>seperate set of</p>
<p class="bold">paragraphs</p></div>
<ul><li>List</li>
<li>of</li>
<li>things</li></ul>
<dl><dt>title</dt><dd>description</dd>
<dt>another title</dt><dd>another description</dd>
<dt>nested list</dt><dd><ul><li>one</li>
<li>two</li></ul></dd></dl>


In [7]:
# Render the HTML in the Notebook
print_html(html.draw())

The HTML object `__repr__()` will return the entire object construction as a string:

In [8]:
print(html)

Container_(H('Header 1', size=1), P('paragraph'), Div(P('seperate set of'), P('paragraphs', class_='bold')), UlList('List', 'of', 'things'), DlDict({'title': 'description', 'another title': 'another description', 'nested list': UlList('one', 'two')}))


Allowing (but not advising) you to `exec` that string to build a replica HTML object:

In [9]:
exec_str = html.__repr__()
exec('exec_rtn = '+exec_str)
print(exec_rtn.draw())

<h1 size="1">Header 1</h1>
<p>paragraph</p>
<div><p>seperate set of</p>
<p class="bold">paragraphs</p></div>
<ul><li>List</li>
<li>of</li>
<li>things</li></ul>
<dl><dt>title</dt><dd>description</dd>
<dt>another title</dt><dd>another description</dd>
<dt>nested list</dt><dd><ul><li>one</li>
<li>two</li></ul></dd></dl>


## Customisation

### Subclassing

The HTML base class (`bootlets.html_base.Base`), which each `bootlets.html` class is subclassed from, offers several attributes to be overwritten to allow more detailed HTML tag definitions:
 - `Base.defaults` - default: `{}` - a dictionaty of default `kwargs`
 - `Base.funcs` - default: `[]` -  a list of strings, which represent names of methods  
 - `Base._tag` - default: `''` - a string of the HTML tag (e.g. 'div')  
 - `Base._block` - default: `'<{tag}{classes}>{content}</{tag}>'` - a string to construct the HTML. Tokens are replaced by:
    - `kwargs`, generated by `Template.get_kwargs()`, returns combined dict of `defaults` and instanciated `**kwargs`, keys in `defaults` that start with `_` are skipped.
    - the functions listed in `funcs`, 
    - `tag`
    - `content`, generated by `Template.get_content()`, returns `\n` join of `*args`, trying to `draw()` each `arg`   


#### Examples

In [10]:
class StyledDiv(Div):
    defaults = {'class_': 'styled'}
    
StyledDiv('some content').draw()

'<div class="styled">some content</div>'

In [11]:
class UsersLink(A):
    defaults = {'href': '#Users'}

UsersLink('users').draw()

'<a href="#Users">users</a>'

In [12]:
class AppendP(P):
    funcs = ['do_append']
    _block = '<{tag}>{do_append}</{tag}>'
    
    def do_append(self):
        return '\n'.join(f'({i+1}) '+ arg for i, arg in enumerate(self.args))

AppendP('1', '2', '3').draw()

'<p>(1) 1\n(2) 2\n(3) 3</p>'

### Calling Object

Alternatively, you can instanciate a `bootlets.html` class then call the object with new `*args` and/or `**kwargs`.

When you call a `bootlets.html` object passing `*args` or `**kwargs`, a new object of the parent class is instanciated, updating any new parameters. Passing any `*args` replace any `*args` of the original object. Passing new `**kwargs` update the dictionary of the original object.

In [13]:
# Create parent HTML object
p = P('original content', class_='styleA')
p.draw()

'<p class="styleA">original content</p>'

In [14]:
# Call with new *args (i.e. replace content)
p('new content').draw()

'<p class="styleA">new content</p>'

In [15]:
# Call with new **kwargs
p(class_='styleB').draw()

'<p class="styleB">original content</p>'

## Boots

`bootlets.boots`, excluding the utility `Container` class, are typically helper classes to draw [Bootstrap](https://getbootstrap.com/) components

In [16]:
from bootlets.boots import *

In [17]:
# https://getbootstrap.com/docs/5.1/components/badge/
Badge("example").draw()

'<span class="badge badge-primary">example</span>'

In [18]:
# https://getbootstrap.com/docs/5.1/components/breadcrumb/
Breadcrumb(
    BreadcrumbItem("item 1"),
    BreadcrumbItem("item 2"),
    BreadcrumbItem("item 3")
).draw()

'<nav aria-label="breadcrumb"><ol class="breadcrumb"><li class="breadcrumb-item"><li class="breadcrumb-item">item 1</li></li>\n<li class="breadcrumb-item"><li class="breadcrumb-item">item 2</li></li>\n<li aria-current="page" class="breadcrumb-item active"><li class="breadcrumb-item">item 3</li></li></ol></nav>'

In [19]:
# https://getbootstrap.com/docs/5.1/components/card/
Card(
    CardHeader("Card Header"),
    CardBody("This is some bootstrap card body content!"),
    CardFooter("Card footer")
).draw()

'<div class="card mb-3"><h4 class="card-header">Card Header</h4>\n<div class="card-body">This is some bootstrap card body content!</div>\n<div class="card-footer">Card footer</div></div>'